In [1]:
import cv2
import numpy as np
import pandas as pd

In [2]:
def get_image_from_file(file_name):
	img = cv2.imread(file_name)
	maskim = cv2.inRange(img, (0, 0, 250), (0, 0, 255))
	img[maskim == 0] = [255, 255, 255]
	# img = cv2.GaussianBlur(img, (5, 5), 0)
	return img

In [3]:
flechas = [get_image_from_file('marcas-capturasStage/Flecha1.png'),
           get_image_from_file('marcas-capturasStage/Flecha2.png'),
           get_image_from_file('marcas-capturasStage/Flecha3.png'),
           get_image_from_file('marcas-capturasStage/Flecha4.png')]

man = [get_image_from_file('marcas-capturasStage/man-1.png'),
       get_image_from_file('marcas-capturasStage/man-2.png'),
       get_image_from_file('marcas-capturasStage/man-3.png'),
       get_image_from_file('marcas-capturasStage/man-4.png'),
       get_image_from_file('marcas-capturasStage/man-5.png'),
       get_image_from_file('marcas-capturasStage/man-6.png'),
       get_image_from_file('marcas-capturasStage/man-7.png')]

stairs = [get_image_from_file('marcas-capturasStage/stairs-1.png'),
          get_image_from_file('marcas-capturasStage/stairs-2.png'),
          get_image_from_file('marcas-capturasStage/stairs-3.png'),
          get_image_from_file('marcas-capturasStage/stairs-4.png'),
          get_image_from_file('marcas-capturasStage/stairs-5.png'),
          get_image_from_file('marcas-capturasStage/stairs-6.png'),
          get_image_from_file('marcas-capturasStage/stairs-7.png')]

telephone = [get_image_from_file('marcas-capturasStage/telephone-1.png'),
             get_image_from_file('marcas-capturasStage/telephone-2.png'),
             get_image_from_file('marcas-capturasStage/telephone-3.png'),
             get_image_from_file('marcas-capturasStage/telephone-4.png'),
             get_image_from_file('marcas-capturasStage/telephone-5.png'),
             get_image_from_file('marcas-capturasStage/telephone-6.png'),
             get_image_from_file('marcas-capturasStage/telephone-7.png')]

woman = [get_image_from_file('marcas-capturasStage/woman-1.png'),
         get_image_from_file('marcas-capturasStage/woman-2.png'),
         get_image_from_file('marcas-capturasStage/woman-3.png'),
         get_image_from_file('marcas-capturasStage/woman-4.png'),
         get_image_from_file('marcas-capturasStage/woman-5.png'),
         get_image_from_file('marcas-capturasStage/woman-6.png'),
         get_image_from_file('marcas-capturasStage/woman-7.png')]

# cv2.imshow('Sample', flechas[0])
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [4]:
def extract_features(img):
	img = cv2.cvtColor(img.copy(), cv2.COLOR_RGBA2GRAY)
	img = img - 255

	cont_list_im, hier = cv2.findContours(img, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

	# cont_list_im = np.array(cont_list_im)
	# cont_list_im = cont_list_im[np.array([cv2.contourArea(conto) for conto in cont_list_im]) > 10]
	orb = cv2.ORB_create()
	ellip = cv2.fitEllipse(cont_list_im[0])
	cen, ejes, angulo = np.array(ellip[0]), np.array(ellip[1]), ellip[2]
	kp = cv2.KeyPoint(cen[0], cen[1], np.mean(ejes) * 1.3, angulo - 90)
	lkp, des = orb.compute(img, [kp])

	return cont_list_im, ellip, lkp, des

In [5]:
sample = flechas[0].copy()

contours, ellipse, lkeyp, dess = extract_features(sample)

for cont in contours:
	cv2.drawContours(sample, cont, -1, (np.random.randint(0, 255), np.random.randint(0, 255), 0), 3)

cv2.ellipse(sample, ellipse, (255, 0, 0), 3)
cv2.drawKeypoints(sample, lkeyp, sample, (0, 255, 0), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv2.imshow('sample', sample)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:
columns = ['area', 'momentx', 'momenty', 'label', 'm00', 'm10', 'm01', 'm20', 'm11', 'm02', 'm30', 'm21', 'm12', 'm03',
           'mu20', 'mu11', 'mu02', 'mu30', 'mu21', 'mu12', 'mu03', 'nu20', 'nu11', 'nu02', 'nu30', 'nu21', 'nu12',
           'nu03']
df = pd.DataFrame(columns=columns)

for arrow in flechas:
	contours, ellipse, lkeyp, dess = extract_features(arrow.copy())
	M_1 = cv2.moments(contours[0])

	if M_1["m00"] == 0: M_1["m00", "m01"] = 1
	x = int(M_1["m10"] / M_1["m00"])
	y = int(M_1["m01"] / M_1["m00"])
	data = {'momentx': x, 'momenty': y, 'area': cv2.contourArea(contours[0]), 'label': 0}
	data.update(M_1)
	df = df.append(data, ignore_index=True)

for m in man:
	contours, ellipse, lkeyp, dess = extract_features(m)
	M_1 = cv2.moments(contours[0])
	if M_1["m00"] == 0: M_1["m00", "m01"] = 1
	x = int(M_1["m10"] / M_1["m00"])
	y = int(M_1["m01"] / M_1["m00"])
	data = {'momentx': x, 'momenty': y, 'area': cv2.contourArea(contours[0]), 'label': 1}
	data.update(M_1)
	df = df.append(data, ignore_index=True)

for stair in stairs:
	contours, ellipse, lkeyp, dess = extract_features(stair)
	M_1 = cv2.moments(contours[0])
	if M_1["m00"] == 0: M_1["m00", "m01"] = 1
	x = int(M_1["m10"] / M_1["m00"])
	y = int(M_1["m01"] / M_1["m00"])
	data = {'momentx': x, 'momenty': y, 'area': cv2.contourArea(contours[0]), 'label': 2}
	data.update(M_1)
	df = df.append(data, ignore_index=True)

for t in telephone:
	contours, ellipse, lkeyp, dess = extract_features(t)
	M_1 = cv2.moments(contours[0])
	if M_1["m00"] == 0: M_1["m00", "m01"] = 1
	x = int(M_1["m10"] / M_1["m00"])
	y = int(M_1["m01"] / M_1["m00"])
	data = {'momentx': x, 'momenty': y, 'area': cv2.contourArea(contours[0]), 'label': 3}
	data.update(M_1)
	df = df.append(data, ignore_index=True)

for w in woman:
	contours, ellipse, lkeyp, dess = extract_features(w)
	M_1 = cv2.moments(contours[0])
	if M_1["m00"] == 0: M_1["m00", "m01"] = 1
	x = int(M_1["m10"] / M_1["m00"])
	y = int(M_1["m01"] / M_1["m00"])
	data = {'momentx': x, 'momenty': y, 'area': cv2.contourArea(contours[0]), 'label': 4}
	data.update(M_1)
	df = df.append(data, ignore_index=True)

In [7]:
df.describe()

,area,momentx,momenty,label,m00,m10,m01,m20,m11,m02,...,mu21,mu12,mu03,nu20,nu11,nu02,nu30,nu21,nu12,nu03
count,32.000000,32.000000,32.000000,32.000000,32.000000,3.200000e+01,3.200000e+01,3.200000e+01,3.200000e+01,3.200000e+01,...,3.200000e+01,3.200000e+01,3.200000e+01,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000
mean,2857.656250,282.593750,201.593750,2.187500,2857.656250,8.876964e+05,6.951191e+05,3.324249e+08,2.271086e+08,1.869751e+08,...,-6.536763e+06,-1.352379e+06,1.369597e+07,0.184621,-0.001310,0.428201,0.005652,-0.001127,0.000615,-0.046607
std,2070.446252,140.124106,83.851197,1.354503,2070.446252,8.858479e+05,7.036689e+05,4.283989e+08,2.771426e+08,2.361962e+08,...,3.121140e+07,1.626184e+07,3.569381e+07,0.188053,0.210676,0.525019,0.065687,0.028391,0.031563,0.134970
min,199.000000,84.000000,63.000000,0.000000,199.000000,3.223467e+04,1.635867e+04,4.727332e+06,2.651370e+06,1.406871e+06,...,-1.282677e+08,-5.800254e+07,-2.969711e+07,0.025802,-0.850650,0.025224,-0.073925,-0.100359,-0.080402,-0.408808
25%,1117.000000,157.750000,139.750000,1.000000,1117.000000,2.667534e+05,1.877481e+05,3.896159e+07,4.413326e+07,4.080131e+07,...,-1.226389e+06,-2.044573e+06,-7.091393e+05,0.074954,-0.094131,0.149522,-0.012984,-0.004736,-0.009929,-0.028957
50%,2422.000000,263.000000,193.000000,2.000000,2422.000000,5.028213e+05,4.337213e+05,1.245947e+08,1.034824e+08,8.403893e+07,...,9.851255e+04,-1.470404e+04,1.499999e+06,0.144233,0.009030,0.180079,-0.000544,0.001522,-0.000963,0.007107
75%,4283.750000,388.000000,273.250000,3.000000,4283.750000,1.474020e+06,1.032961e+06,4.810383e+08,3.435361e+08,2.476583e+08,...,1.490326e+06,1.776654e+06,1.191353e+07,0.223024,0.123999,0.274381,0.003980,0.012184,0.011802,0.018213
max,8085.500000,534.000000,344.000000,4.000000,8085.500000,3.416661e+06,2.763516e+06,1.767303e+09,1.099436e+09,9.552476e+08,...,3.065863e+07,3.603446e+07,1.466715e+08,1.011823,0.497413,1.618738,0.298467,0.081839,0.085821,0.200543


In [10]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier()
clf.fit(df.drop(columns=['label']), df['label'])

DecisionTreeClassifier()

In [11]:
import joblib

joblib.dump(clf, 'rec_marcas_model.jl')

['rec_marcas_model.jl']

In [10]:
vid = cv2.VideoCapture('proyectoRobotica-1920-line1-video-2020-04-20.avi')
lables = ['flecha', 'man', 'stair', 'telephone', 'woman']

width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
height =  int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video = cv2.VideoWriter('videoMarcas.avi', fourcc, 30, (width, height))

while True:
	frameo = vid.read()[1]
	frame = frameo.copy()
	maskframe = cv2.inRange(frame, (0, 0, 100), (50, 50, 255))
	frame[maskframe == 0] = [255, 255, 255]

	try:
		contours, ellipse, lkeyp, dess = extract_features(frame)
		if len(contours) != 1:
			contours = contours[0] if cv2.contourArea(contours[0]) > cv2.contourArea(contours[1]) else contours[1]
		M_1 = cv2.moments(contours[0])

		if M_1["m00"] == 0: M_1["m00", "m01"] = 1
		x = int(M_1["m10"] / M_1["m00"])
		y = int(M_1["m01"] / M_1["m00"])
		data = {'momentx': x, 'momenty': y, 'area': cv2.contourArea(contours[0])}
		data.update(M_1)
		odf = pd.DataFrame(columns=columns)
		odf.drop(columns=['label'], inplace=True)
		odf = odf.append(data, ignore_index=True)
		predicted = clf.predict(odf)
		cv2.putText(frame, lables[int(predicted)], (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
		cv2.putText(frameo, lables[int(predicted)], (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
		# print(lables[int(predicted)])
	except IndexError as e:
		cv2.putText(frameo, 'Nothing', (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
	except Exception as e:
		cv2.putText(frameo, 'Nothing', (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

	cv2.imshow('frameo', frameo)
	cv2.imshow('frame', frame)

	video.write(np.uint8(frameo))
	video.write(np.uint8(frameo))
	video.write(np.uint8(frameo))

	cv2.destroyAllWindows()

	if cv2.waitKey(55) & 0xFF == ord('q'):
		break

video.release()
cv2.destroyAllWindows()

man
man
man
man
man
man
man
man
man
man
man
man
woman
woman
flecha
woman
woman
woman
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
woman
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
woman
woman
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
telephone
telephone
telephone
telephone
telephone
telephone
telephone
telephone
telephone
telephone
telephone
telephone
man
man
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
flecha
stair
stair
stair
stair
stair
stair
stair
stair
stair
stair
stair
stair
stai

AttributeError: 'NoneType' object has no attribute 'copy'